# Тестовые данные

In [2]:
# (input, output)
test_1 = ([[3,2,1],[1,7,6],[2,7,7]], 1)
test_2 = ([[3,1,2,2],[1,4,4,5],[2,4,2,2],[2,4,2,2]], 3)
test_3 = ([[3,2,1,1],[1,7,6,6],[2,7,7,7],[2,7,7,7]], 2)
test_4 = ([[1, 1], [1, 1]], 4)

# Кубический вариант

Cначала я решила додумать решение по подсказке, но куб остался :(

In [3]:
def count_equal_rowcol_pairs(grid):
    '''Сложность: O(n^3)'''
    
    n = len(grid)
    pairs_counter = 0
    
    # проходимся по строкам
    for row in grid:
        # для текущей строки проходимся по всем колонкам
        for j in range(n):
            for i in range(n):
                # как только находим неравные элементы - бросаем сравниваемую колонку
                if row[i] != grid[i][j]:
                    break
                # если проитерировались по всей колонке - значит найдена пара, т.к. все соответствующие элементы равны
                if i == n-1:
                    pairs_counter += 1
    
    return pairs_counter

In [9]:
assert count_equal_rowcol_pairs(test_1[0]) == test_1[1]
assert count_equal_rowcol_pairs(test_2[0]) == test_2[1]
assert count_equal_rowcol_pairs(test_3[0]) == test_3[1]
assert count_equal_rowcol_pairs(test_4[0]) == test_4[1]

Потом у меня было два промежуточных решения, мне казалось, что я нашла O(n^2), но когда доходило дело до реализации - я снова упиралась в какой-нибудь подкапотный O(n) и куб возвращался.

Я долго думала, как мне избавиться от куба и прийти к квадрату. Либо мне нужно как-то сравнивать списки, не перебирая n элементов, либо оставить сравнение списков за O(n), но оно должно быть внутри одного цикла, а не вложенного. 

# Квадратичный вариант

Мой мозг зацепился за сравнение списков за O(n) и вспомнил префиксное дерево (trie), в котором поиск слова происходит за O(n). Я прикинула, а что если я добавлю строки в trie, а потом буду искать колонки в этом trie? И, кажется, получился заветный квдрат :)

P.S.: кстати, с trie я столкнулась впервые, работая над дипломом в бакалавриате - на этой структуре сделан mystem от Яндекса :)

In [12]:
def count_equal_rowcol_pairs(grid):
    '''Сложность: O(n^2)'''
    
    n = len(grid)
    
    # добавление и поиск в trie: O(n)
    # trie[0] - переходы к другим вершинам
    # trie[1] - сколько раз дошли до этой вершины
    #   для всего, что не лист - 1,
    #   а для листа - этот счетчик плюсуется, т.к. нам нужно считать попадания в лист при построении дерева (кол-во одинаковых строк)
    trie = [dict(), 1]

    # строим trie из строк: O(n^2)
    for row in grid:
        current_node = trie
        for i, item in enumerate(row):
            try:
                current_node = current_node[0][item]
                # дошли до конца слова, которое уже было добавлено в дерево, поэтому плюсуем счетчик кол-ва таких слов
                if i == n-1:
                    current_node[1] += 1
            except KeyError:
                current_node[0][item] = [dict(), 1]
                current_node = current_node[0][item]

    # генерим колонки: O(n^2)
    columns = []
    for i in range(n):
        column = []
        for j in range(n):
            column.append(grid[j][i])
        columns.append(column)
    
    # обходм колонки и ищем их в trie из строк: O(n^2)
    pairs_counter = 0
    for column in columns:
        current_node = trie
        for i, item in enumerate(column):
            try:
                current_node = current_node[0][item]
                if i == n-1:
                    pairs_counter += current_node[1]
            except KeyError:
                break

    return pairs_counter

In [13]:
assert count_equal_rowcol_pairs(test_1[0]) == test_1[1]
assert count_equal_rowcol_pairs(test_2[0]) == test_2[1]
assert count_equal_rowcol_pairs(test_3[0]) == test_3[1]
assert count_equal_rowcol_pairs(test_4[0]) == test_4[1]